# M3 Project

In this project, you will compare the performance of uninformed and informed search algorithms for the 4-sided dominoes problem. You will be given N<sup>2</sup> tiles (N = {2,3,4}) and will be asked to arrange them in an NxN grid in a way that adjacent tiles have the same number in their neighboring sides. The tiles cannot be rotated. See the example below for N=3.

<img src="m3project.png" width="400"/>

An initial version of the code with the problem specification (below) and a report template (at the bottom) are available in this notebook. Deliverables are the final code (non-functioning code is worth 0 points) and the comparison report.

Solve the task above using:
- one uninformed search algorithm of your choice (20pts)
- one informed search algorithm of your choice (20pts)

For your solution, describe the:
- search state space (10pts)
- successor function (10pts)
- heuristic function for the informed search (10pts)

Run each algorithm with at least 10 different initial states for each value of N, and compare the performance of the chosen algorithms for different puzzle sizes in terms of:
- average number of expanded states (15pts)
- success rate (15pts)

You are free to set a maximum number of expanded states for each algorithm, and return failure in case this number is reached.

# Implementation

You are free to change the code below as needed.

In [3]:
import random
import heapq
import numpy as np

In [4]:
class Domino():
    """
    Implementation of a 4-sided domino tile.

    Methods
    -------
    is_above(other)
        Checks of the tile is above the other tile.
    is_under(other)
        Checks of the tile is under the other tile.
    is_on_the_left_of(other)
        Checks of the tile is on the left of the other tile.
    is_on_the_right_of(other)
        Checks of the tile is on the right of the other tile.
    """
    def __init__(self, top: int, right: int, bottom: int, left: int):
        assert isinstance(top, int) and isinstance(right, int) and isinstance(bottom, int) and isinstance(left, int), "Invalid tile value!"
        self.top = top
        self.right = right
        self.bottom = bottom
        self.left = left

    def is_above(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.bottom == other.top
    
    def is_under(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.top == other.bottom
    
    def is_on_the_left_of(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.right == other.left
    
    def is_on_the_right_of(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.left == other.right
    
#     less than function: Equates if one domino object is less than another domino
    def __lt__(self, other):
        return(self.top, self.right, self.bottom, self.left) < (other.top, other.right, other.bottom, other.left)

In [5]:
"""
This implementation is provided as a starting point. Feel free to change it as needed.
"""
class FourDominoes():
    """
    Implementation of the 4-sided dominoes puzzle.

    Methods
    -------
    show()
        Visualize the current state.
    move(action)
        Apply an action to the current state.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
    """

    def __init__(self, N, state=None):
        """
        Parameters
        ----------
        N
            Grid size (puzzle contains N^2 tiles)
        state
            Initial state configuration. If None is provided, a random one is created.
        """
        assert N >= 2 and N <= 4, "Invalid grid size!"
        self.N = N

        if state is not None:
            assert len(state) == self.N, "Invalid state size!"
            for row in state:
                assert len(row) == self.N, "Invalid state size!"
                for tile in row:
                    assert isinstance(tile, Domino), "Invalid state type!"
            self.state = state
        else:
            self.state = self.__get_random_state()
    
    def __get_random_state(self):
        """
        Generates a random puzzle configuration (grid of dominoes)

        Return
        ----------
        tuple
            A tuple describing a unique puzzle configuration.
        """
        temp = []
        for i in range(self.N):
            for j in range(self.N):
                domino = Domino(
                    random.randint(1,9) if i == 0 else temp[(i-1)*self.N+j].bottom, # top
                    random.randint(1,9),                                            # right
                    random.randint(1,9),                                            # bottom
                    random.randint(1,9) if j == 0 else temp[i*self.N+j-1].right     #left
                )
                temp.append(domino)
        random.shuffle(temp) # if you comment this line, the state will be a final state (solution)
        return tuple(tuple(temp[i*self.N:(i+1)*self.N]) for i in range(self.N))

    def show(self):
        """
        Prints the current state.
        """
        #line
        print('╔', end='')
        for i in range(self.N):
            print('═══', end='╦' if i < self.N-1 else '╗\n')
        for i in range(self.N):
            # first line of tile
            print('║', end='')
            for j in range(self.N):
                print('╲{}╱║'.format(self.state[i][j].top), end='' if j < self.N-1 else '\n')
            # second line of tile
            print('║', end='')
            for j in range(self.N):
                print('{}╳{}║'.format(self.state[i][j].left, self.state[i][j].right), end='' if j < self.N-1 else '\n')
            # third line of tile
            print('║', end='')
            for j in range(self.N):
                print('╱{}╲║'.format(self.state[i][j].bottom), end='' if j < self.N-1 else '\n')
            # line
            print('╠' if i < self.N-1 else '╚', end='')
            for j in range(self.N):
                print('═══', end='╬' if i < self.N-1 and j < self.N-1 else '╩' if i == self.N-1 and j < self.N-1 else '╣\n' if i < self.N-1 else '╝\n')

    def move(self, action):
        """
        Uses a given action to update the current state of the puzzle. Assumes the action is valid.

        Parameters
        ----------
        action
            Tuple with coordinates of two tiles to be swapped ((row1,col1), (row2,col2))
        """
        assert len(action) == 2 and all(len(coord) == 2 for coord in action) and all(isinstance(x, int) and x >= 0 and x < self.N for coord in action for x in coord), "Invalid action!"
        (r1, c1), (r2, c2) = action
        temp = [list(row) for row in self.state]
        temp[r1][c1], temp[r2][c2] = temp[r2][c2], temp[r1][c1]
        self.state = tuple(tuple(x) for x in temp)

    def successor(self, state):
        """
        Finds the list of successors for a given state.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        list
            A list of pairs (action,state) with all states that can be reached from the given state with a single action.
        """

        successors = []
        for i in range(self.N*self.N):
            r1 = i//self.N
            c1 = i%self.N
            for j in range(i+1, self.N*self.N):
                r2 = j//self.N
                c2 = j%self.N
                action = ((r1,c1), (r2,c2))
                temp_state = [list(row) for row in state]
                temp_state[r1][c1], temp_state[r2][c2] = temp_state[r2][c2], temp_state[r1][c1]
                successors.append((action, tuple(tuple(row) for row in temp_state)))
        return successors

    def goal_test(self, state):
        """
        Checks if the given state is a goal state.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        bool
             True if the given state is a goal state, and False otherwise.
        """

        for i in range(self.N):
            for j in range(self.N):
                if i > 0 and not state[i][j].is_under(state[i-1][j]):
                    return False
                if j > 0 and not state[i][j].is_on_the_right_of(state[i][j-1]):
                    return False
        return True

In [6]:
"""
Experimental functions:
    BFS: performs bfs on the fourdominoes problem with max expanded states of 10000
    
    heuristic_misplaced_tiles: estimates cost of reaching goal state from a given state in the context 
    of the A* search algorithm
    
    a_star_search: performs A* search on the fourdominoes problem with max expanded states of 10000
"""

def bfs(problem, max_expanded_states=10000):
    frontier = [(problem.state, [problem.state])]
    explored = set()
    expanded_states = 0

    while frontier:
        node, path = frontier.pop(0)
        if problem.goal_test(node):
            return path, expanded_states
        explored.add(tuple(tuple(tile for tile in row) for row in node))
        for action, successor in problem.successor(node):
            if tuple(tuple(tile for tile in row) for row in successor) not in explored:
                frontier.append((successor, path + [successor]))
        expanded_states += 1
        if expanded_states >= max_expanded_states:
            return None, expanded_states
    return None, expanded_states

def heuristic_misplaced_tiles(state):
    misplaced_edges = 0
    misplaced_positions = 0
    N = len(state)
    
    for i in range(N):
        for j in range(N):
            tile = state[i][j]
            if i > 0 and tile.top != state[i-1][j].bottom:
                misplaced_edges += 1
            if j > 0 and tile.left != state[i][j-1].right:
                misplaced_edges += 1
            if i < N - 1 and tile.bottom != state[i+1][j].top:
                misplaced_edges += 1
            if j < N - 1 and tile.right != state[i][j+1].left:
                misplaced_edges += 1
            correct_top = (i == 0 or state[i-1][j].bottom == tile.top)
            correct_left = (j == 0 or state[i][j-1].right == tile.left)
            correct_bottom = (i == N-1 or state[i+1][j].top == tile.bottom)
            correct_right = (j == N-1 or state[i][j+1].left == tile.right)
            if not (correct_top and correct_left and correct_bottom and correct_right):
                misplaced_positions += 1

    return misplaced_edges + misplaced_positions

def greedy_best_first_search(problem, heuristic, max_explored_states=10000):
    start_state = problem.state
    frontier = [(heuristic(start_state), start_state)]
    frontier_set = {start_state}
    heapq.heapify(frontier)
    explored = set()
    origin = {}
    expanded_states = 0
    
    while frontier:
        _, current_state = heapq.heappop(frontier)
        frontier_set.remove(current_state)
        if problem.goal_test(current_state):
            return reconstruct_path(origin, current_state), expanded_states
        expanded_states += 1
        if expanded_states >= max_explored_states:
            return None, expanded_states
        explored.add(current_state)
        for action, neighbor in problem.successor(current_state):
            if neighbor not in explored and neighbor not in frontier_set:
                origin[neighbor] = current_state
                h_score = heuristic(neighbor)
                heapq.heappush(frontier, (h_score, neighbor))
                frontier_set.add(neighbor)
    return None, expanded_states

def reconstruct_path(origin, current_state):
    """
    Reconstructs path from inital state to goal state by tracing back from the goal state to the initial state using the origin dictionary.
    The dictionary maps each state to its predecessor state, allowing the path to be traced in reverse.
    
    Input: current state, the current state explored
           origin, the origin dictionary
           
    Output: a list containing the sequence of states from the initial to goal state
    """
    path = [current_state]
    while current_state in origin:
        current_state = origin[current_state]
        path.insert(0, current_state)
    return path

In [ ]:
"""
run_experiment function:
    input: N_values, number of trials to run
    output: average expanded states and success rate
"""
def run_experiments(N_values, num_trials):
    results = {N: {'bfs': {'expanded_states': [], 'success_rate': 0},
                   'gbfs': {'expanded_states': [], 'success_rate': 0}}
               for N in N_values}

    for N in N_values:
        print(f"Running experiments for N={N}")
        for _ in range(num_trials):
            puzzle = FourDominoes(N)
            #UNCOMMENT BELOW LINE TO SEE INITIAL PUZZLE STATE FOR EACH TRIAL!
            # puzzle.show()

            # BFS
            solution, expanded_states = bfs(puzzle)
            if solution:
                results[N]['bfs']['success_rate'] += 1
                results[N]['bfs']['expanded_states'].append(expanded_states)
            else:
                results[N]['bfs']['expanded_states'].append(expanded_states)

            # Greedy Best-First Search
            solution, expanded_states = greedy_best_first_search(puzzle, heuristic_misplaced_tiles)
            if solution:
                results[N]['gbfs']['success_rate'] += 1
                results[N]['gbfs']['expanded_states'].append(expanded_states)
            else:
                results[N]['gbfs']['expanded_states'].append(expanded_states)

        # calc avg expanded states and success rate
        results[N]['bfs']['average_expanded_states'] = np.mean(results[N]['bfs']['expanded_states'])
        results[N]['bfs']['success_rate'] /= num_trials

        results[N]['gbfs']['average_expanded_states'] = np.mean(results[N]['gbfs']['expanded_states'])
        results[N]['gbfs']['success_rate'] /= num_trials

    return results

# Trial settings and variables
N_values = [2, 3, 4]  # puzzle sizes to test
num_trials = 10       # number of trials for each puzzle size
results = run_experiments(N_values, num_trials)

# PRINT RESULTS!
for N in N_values:
    print(f"Results for N={N}:")
    print(f"BFS - Average Expanded States: {results[N]['bfs']['average_expanded_states']}, Success Rate: {results[N]['bfs']['success_rate']}")
    print(f"GBFS - Average Expanded States: {results[N]['gbfs']['average_expanded_states']}, Success Rate: {results[N]['gbfs']['success_rate']}")


Running experiments for N=2
Running experiments for N=3
Running experiments for N=4


In [ ]:
# Report template

## Solution description

Methods implemented: 
For this project, I implemented two search algorithms. Breadth-First Search (BFS) and A* Search.
Both algorithms aim to find a solution path from the initialk state to the goal state of the Four Dominoes puzzle. 
### Search space

Format, size, start, goal, etc.

The search space for the Four Dominnoes puzzle consits of all possible arrangements of dominoes on a square grid.
The size of the grid varies depending on the puzzle size, with dimensions ranging from 2x2 to 4x4. The start state 
represents thje initial arrangement of dominoes randomly generated within the constraints of the puzzle size.
The goal state is reached when all dominoes are correctly placed relative to their adjacent tiles, satisfying the puzzle's constraints.

### Successor function

How it works? Average number of successors? Etc. 
The successor function generates all poissible successor states from a given state by swapping the positions of two dominoes. 
It iterates over each domino in the grid and considers all valid swaps with other dominoes. This ensures that all potential 
configurations are explored. It works by iterating through all pairs of tiles in the NxN grid and generating anew state for
each swap of two tiles. This is done for all possible pairs of tiles in the grid, ensuring a comprehensive exploration of the 
state space. The average number of successors depend on the number of tiles in the grid, N^2. For a grid of size N, the total 
number of possible swaps would be equivalent to (N^2 * (N^2-1))/2. However, since the expirement only considers valid swaps, the 
actual number of successors is loweer. For example, for a 2x2 grid, there is only 1 possible swap to be made. For a 3x3 grid, there are
(9x8)/2 possible swaps, and for a 4x4 grid, there are (16x15)/2 possible swaps.  Some states that would be impossible to each are dominoes that are
in the same positon, as swappinga  domino with itself does not generate a new state and isn't considered.

### Heuristic function

How it works? Admissible? 

The huristic function I used in this iteration of the experiment was derived from my first attempt at the project.
The function estimates the cost to reach the goal state from the current state by counting the number of mismatched 
edges and incorrectly placed dominoes. This heuristic is not admissible, however, the greedy best search algorithm 
does not require an admissible heuristic for it to function properly. To check for mismatched edges, the function 
iterates through each tile in the grid and checks if the edges match with the adjacent tiles. For each mismatch, it 
increments a counter. The second part of my heuristic function checks if the dominoes are correctly placed by verifying 
sides of each tile. The total heuristic value is the sum of mismatched edges and incorrectly placed dominoes. 

## Experimental results

Below are the results from running the expirement 10 times for both BFS and GBFS twice.

Run #1

Results for N=2:
BFS - Average Expanded States: 17.5, Success Rate: 1.0
GBFS - Average Expanded States: 3.2, Success Rate: 1.0
Results for N=3:
BFS - Average Expanded States: 10000.0, Success Rate: 0.0
GBFS - Average Expanded States: 35.1, Success Rate: 1.0
Results for N=4:
BFS - Average Expanded States: 10000.0, Success Rate: 0.0
GBFS - Average Expanded States: 974.2, Success Rate: 1.0
 
Run #2

